In [ ]:
import os
from odf.opendocument import load
from odf.table import Table
from odf.element import Element
import pandas as pd
from pynxtools_apm.examples.oasisb_utils import (
    is_valid_alpha3,
    snake_case_to_camel_case,
)
import bibtexparser

# TODO parameterize
src_directory = "CHANGEME".replace("/", os.sep)

### Reformat the ods as that had successively grown becoming unmanagable, split these sheets up and standardize

* For each project sheet we do not want to have blank rows to concatenate the project sub-directory, sheet row_id for making entries and uploads be unique and descriptive.*
* sheets originally where all in one spreadsheet, which grew and become difficult to navigate, also not the typical use case, multiple people would puzzle together knowledge<br>
* Therefore, we disentangle here the original spreadsheet and clean it from blanks plus some more harmonization of formatting to reflect newer parsing capabilities for more<br>
  infrequently represented community file formats, typical NOMAD upload then aus_sydney_breen with entries named aus_sydney_breen_{1, 2, ...}

In [ ]:
run_sorting = False
if run_sorting:
    doc = load(f"{src_directory}{os.sep}harvest.examples.14.apm.xls.ods")

    spreadsheet = doc.spreadsheet
    sheets = [
        sheet
        for sheet in spreadsheet.childNodes
        if getattr(sheet, "tagName", "") == "table:table"
    ]
    sorted_sheets = sorted(sheets, key=lambda s: s.getAttribute("name"))
    for sheet in sheets:
        spreadsheet.removeChild(sheet)
    for sheet in sorted_sheets:
        spreadsheet.appendChild(sheet)

    doc.save(f"{src_directory}{os.sep}legacy_data_tmp.ods")

In [ ]:
# get sheet_names, modify these individually and export to project-specific file
# as the initial idea to have the sheets all together was simple when the collection
# was not that large but it becomes more and more impractical to navigate this
# large ODS sheet
doc = load(f"{src_directory}{os.sep}legacy_data_tmp.ods")
spreadsheet = doc.spreadsheet
sheet_names = [
    node.getAttribute("name")
    for node in spreadsheet.childNodes
    if getattr(node, "tagName", "") == "table:table"
]
# print(sheet_names)

Splitting it up makes also sense because legacy data mappings would typically be<br>
contributed efforts by multiple people. Which is what we are here mimicking.

In [ ]:
run_splitting = False
if run_splitting:
    column_header = [
        "str_rraw",
        "rhit_hits",
        "root",
        "pos_epos_apt_ato_csv",
        "rng_rrng_fig_env",
        "hdf_xml_nxs_raw_ops",
    ]
    for sheet_name in sheet_names[1:]:
        if sheet_name != "aaa_legacy_data":
            print(f"Validating {sheet_name}")
            df = pd.read_excel(
                f"{src_directory}{os.sep}legacy_data_tmp.ods",
                sheet_name=sheet_name,
                engine="odf",
            )
            df = df.fillna("")
            df = df[~((df.isna() | (df == "")).all(axis=1))]  # keep non-blank rows only

            # equalize number of columns
            if len(column_header) > df.shape[1]:  # needs adding empty columns
                for idx in range(0, len(column_header) - df.shape[1]):
                    df[f"new_column_{idx}"] = ""
                df.columns = column_header + [
                    f"ignore{col_idx}"
                    for col_idx in range(0, len(column_header) - df.shape[1])
                ]
            elif df.shape[1] > len(column_header):  # needs expanding the column_header
                df.columns = column_header + [
                    f"ignore{col_idx}"
                    for col_idx in range(0, df.shape[1] - len(column_header))
                ]
            else:
                df.columns = column_header + [
                    f"ignore{col_idx}"
                    for col_idx in range(0, df.shape[1] - len(column_header))
                ]

            # report issues with incorrectly categorized datasets
            for idx, row in df.iterrows():
                for col_name in df.columns:
                    if row[col_name] != "" and not col_name.startswith("ignore"):
                        if row[col_name][
                            row[col_name].rfind(".") + 1 :
                        ].lower() not in col_name.split("_"):
                            print(
                                f"WARNING, {sheet_name}, {idx}, {col_name}, {row[col_name]}"
                            )
                        # else:
                        #     print(f"{idx}, {col_name}, OK")

            df.to_excel(
                f"{src_directory}{os.sep}data{os.sep}{sheet_name}.ods",
                sheet_name=sheet_name,
                engine="odf",
                index=False,
            )
            del df

### Validate formatting of the legacy data and that sufficient bibliographical metadata is available

Do we have for each sub-directory also a project file and vice versa?

In [ ]:
def get_bibliographical_metadata(
    bib: dict, snake_case_project_name: str
) -> list[str, str]:
    """Get dataset and article citation_key for given project."""
    matching: dict[str, list[str]] = {
        "data": [],
        "paper": [],
    }
    camel_case_project_name = snake_case_to_camel_case(snake_case_project_name)
    for key in bib:
        for prefix, cls in [("D_", "data"), ("A_", "paper")]:
            if key == f"{prefix}{camel_case_project_name}":
                matching[cls].append(key)
    data_article: list[str, str] = ["", ""]
    for idx, cls, entry_type in [
        (0, "data", "an original dataset"),
        (1, "paper", "an original research article"),
    ]:
        if len(matching[cls]) == 0:
            # if cls == "data":
            print(f"ERROR, {snake_case_project_name} has no reference for {entry_type}")
            # print(f"@Misc{{D_{camel_case_project_name}}},\n  author={{}},\n note = {{personal communication}},\n year = {{2024}},\n}},")
        elif len(matching[cls]) > 1:
            print(
                f"WARNING, {snake_case_project_name} has more than one reference for {entry_type}"
            )
        else:
            data_article[idx] = matching[cls][0]
    return data_article

In [ ]:
ods = pd.read_excel(
    f"{src_directory}{os.sep}data{os.sep}aaa_legacy_data.ods",
    sheet_name="aaa_legacy_data",
    engine="odf",
)
with open(f"{src_directory}{os.sep}data{os.sep}aaa_legacy_data.bib") as fp:
    bib = bibtexparser.load(fp).entries_dict

for row in ods.itertuples(index=True):
    if row.parse in (1, 2):
        # print(row.project_name)
        if not is_valid_alpha3(row.project_name[0 : row.project_name.find("_")]):
            print(f"{row.project_name} has invalid country alpha3 code")
            continue
        prefix = f"{src_directory}{os.sep}data{os.sep}"
        if not os.path.isdir(f"{prefix}{row.project_name}"):
            print(f"{row.project_name} has no sub-directory")
            continue
        if not os.path.isfile(f"{prefix}{row.project_name}.ods"):
            print(f"{row.project_name} has no valid ods file")
            continue
        try:
            df = pd.read_excel(
                f"{prefix}{row.project_name}.ods",
                sheet_name=row.project_name,
                engine="odf",
            )
            del df
        except ValueError:
            print(f"{row.project_name} ods sheet has an inconsistent sheet_name")
        data, article = get_bibliographical_metadata(bib, row.project_name)
        # print(f"{row.project_name}, __{data}__, __{article}__")
print("Integriety check performed.")
# for sheet_name in sheet_names[1:]:
#     sub_directory = f"{src_directory}{os.sep}data{os.sep}{sheet_name}"

#     print(sheet_name)
# print(f"{sub_directory} {'OK' if os.path.isdir(sub_directory) else 'MISSING'}")
# if not os.path.isdir(sub_directory):
#     print(f"{sub_directory} MISSING")

### Check that bibliography matches in formatting and content to the legacy data

SnakeCase sub-directory names matching CamelCase citation keys, assure one D_ entry for each sub-directory

In [ ]:
## Fill in manual atom_types for all those datasets for which no associated ranging definitions file was share, e.g., morgado, etc.

In [ ]:
## Nice to have would be to remove duplicated files